In [ ]:
import pandas as pd
import numpy as np
from pygam import GAM, s
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error
import matplotlib.pyplot as plt


In [ ]:
df = pd.read_csv('Q_Donnees.csv')

# Préparation des données
variables_meteo = ['RR', 'TM', 'FFM', 'DXY']
variable_pollution = 'CNO2'
df = df.dropna(subset=variables_meteo + [variable_pollution])
df['AAAAMMJJ'] = pd.to_datetime(df['AAAAMMJJ'], format='%Y-%m-%d', errors='coerce')
# Conversion des dates

df.set_index('AAAAMMJJ', inplace=True)
date_test = df.index.values

print(date_test)
df.head()

In [ ]:
# Filtrage des données pour les années 1995 à 2019 pour l'entraînement
df_train = df[(df.index.year >= 1995) & (df.index.year <= 2019)]

# Filtrage des données pour les années 2020 à 2023 pour le test
df_test = df[(df.index.year >= 2020) & (df.index.year <= 2023)]

# Préparation des données pour le modèle
X_train = df_train[variables_meteo].values
y_train = df_train[variable_pollution].values
X_test = df_test[variables_meteo].values
y_test = df_test[variable_pollution].values
date_test = df_test.index.values

# Normalisation des données
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Ajustement du modèle GAM
gam = GAM(s(0) + s(1) + s(2) + s(3))
gam.fit(X_train, y_train)

# Prédiction sur les données de test
y_test_pred = gam.predict(X_test)

# Calcul des métriques de performance
r2 = r2_score(y_test, y_test_pred)
mse = mean_squared_error(y_test, y_test_pred)
rmse = np.sqrt(mse)
mape = mean_absolute_percentage_error(y_test, y_test_pred)

print(f"R^2: {r2}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"MAPE: {mape}")

# Affichage des résultats
plt.figure(figsize=(10, 5))
plt.plot(date_test, y_test, label='Valeurs Observées', color='blue')
plt.plot(date_test, y_test_pred, label='Valeurs Prédites', color='orange', linestyle='--')
plt.xlabel('Date')
plt.ylabel('Concentration en NO2')
plt.title('Prédiction de la Concentration en NO2 avec GAM pendant la periode du Covid (2020-2023)')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()